In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random
from pyrithmetic import Addition, Task

In [ ]:
addition_tasks = [
    Addition.AddNoCarryTask(difficulty_level=10, num_digits=1),
    Addition.SumExactly10Task(difficulty_level=20),
    Addition.AddSingleDigitTo10Task(difficulty_level=30),
    Addition.AddTwoNumbersBelow10Task(difficulty_level=40),
    Addition.AddNoCarryTask(difficulty_level=50, num_digits=1, second_digits=2),
    Addition.AddSingleDigitToTwoDigitRoundTask(difficulty_level=60),
    Addition.AddSingleDigitToTwoDigitTask(difficulty_level=70),
    Addition.AddNoCarryTask(difficulty_level=80, num_digits=2),
    Addition.AddTwoDigitToTwoDigitTask(difficulty_level=90),
    Addition.AddDoubleDigitToHundredsTask(difficulty_level=100),
    Addition.AddNoCarryTask(difficulty_level=110, num_digits=2, second_digits=3),
    Addition.AddTwoDigitToThreeDigitTask(difficulty_level=120),
    Addition.AddTwoDigitToThreeDigitTaskWithCarry(difficulty_level=130),
    Addition.AddThreeDigitToThreeDigitTask(difficulty_level=140),
]

addition_tasks = sorted(addition_tasks, key=lambda task: task.difficulty_level)

default_progress = {"average_time": 3, "average_result": 0.5, "count": 0}

In [ ]:
def pick_a_task(progress_data):
    tasks = addition_tasks

    probabilities = []
    activated = []
    previous_activated = True
    for task in tasks:
        try:
            progress = progress_data[task.id]
        except KeyError:
            progress = default_progress

        if previous_activated:
            activated.append(True)
            previous_activated = task.has_met_targets(
                progress["average_result"], progress["average_time"]
            )
        else:
            activated.append(False)
            probabilities.append(0.2 * probabilities[-1])
            continue

        probability = task.probability_of_selection(
            progress["average_result"], progress["average_time"]
        )
        probabilities.append(probability)

    # Normalize probabilities to create weights
    total_probability = sum(probabilities)
    weights = [probability / total_probability for probability in probabilities]

    # Perform weighted random selection
    chosen_task = random.choices(tasks, weights=weights)[0]
    task, correct_answer = chosen_task.generate_assignment()

    return chosen_task, chosen_task.id

In [ ]:
def get_odds(level, ref_level):
    odds = 1 - Task.sigmoid(level, shift=ref_level, scale=0.3)
    return odds


fig = plt.figure()
ax = fig.add_subplot()
x = np.linspace(0, 150)
ref_level = 60
y = [get_odds(v, ref_level) for v in x]
ax.plot(x, y)

In [ ]:
def process_task(progress_data, level):
    task, task_type = pick_a_task(progress_data)

    if task_type not in progress_data.keys():
        progress_data[task_type] = default_progress.copy()

    odds = get_odds(task.difficulty_level, level)
    success = random.uniform(0, 1) < odds

    stats = progress_data[task_type]
    stats["count"] += 1
    stats["average_time"] = 0.95 * stats["average_time"] + 0.05 * 2
    stats["average_result"] = 0.95 * stats["average_result"] + 0.05 * success

    progress_data[task_type] = stats

    return progress_data

In [ ]:
difficulties = [task.difficulty_level for task in addition_tasks]


def get_properties(task_id, progress_data):
    try:
        count = progress_data[task_id]["count"]
    except KeyError:
        count = 0
    return count


def get_counts(progress_data, scaled=False):
    counts = np.array(
        [get_properties(task.id, progress_data) for task in addition_tasks]
    )
    if scaled:
        counts = counts / sum(counts)
    return counts


progress_data = {}
for i in range(10000):
    progress_data = process_task(progress_data, 60)
counts_start = get_counts(progress_data)
for i in range(1000):
    progress_data = process_task(progress_data, 60)
counts_end = get_counts(progress_data)
counts = counts_end - counts_start
counts = counts / sum(counts)


fig = plt.figure()
ax = fig.add_subplot()
ax.bar(difficulties, counts, width=10, ec="w")


# progression = []
# iterations = []
# progress_data = {}
# for i in range(10):
#     for j in range(100):
#         progress_data = process_task(progress_data, 1000)
#     counts = get_counts(progress_data)
#     progression.append(counts)
#     iterations.append(i)

# fig = plt.figure()
# ax = fig.add_subplot()
# for i, counts in zip(iterations, progression):
#     ax.plot(difficulties, counts)